In [1]:
import os
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.tensorboard.plugins import projector
import numpy as np
PATH = os.getcwd()

LOG_DIR = os.path.join(PATH, 'mnist-tensorboard','logs')
metadata = os.path.join(LOG_DIR, 'metadata.tsv')

mnist = tf.keras.datasets.mnist

In [2]:
tf.reset_default_graph()

In [3]:
(x_train, y_train),(x_test, y_test) = mnist.load_data()

In [4]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False)
y_test = y_test.reshape(-1,1)
y_test = ohe.fit_transform(y_test)
x_test = x_test.reshape(-1,784)

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [5]:
images = tf.Variable(x_test, name='images')
#def save_metadata(file):
with open(metadata, 'w') as metadata_file:
    for row in range(10000):
        c = np.nonzero(y_test)[1:][0][row]
        metadata_file.write('{}\n'.format(c))


with tf.Session() as sess:
    saver = tf.train.Saver([images])

    sess.run(images.initializer)
    saver.save(sess, os.path.join(LOG_DIR, 'images.ckpt'))

    config = projector.ProjectorConfig()
    # One can add multiple embeddings.
    embedding = config.embeddings.add()
    embedding.tensor_name = images.name
    # Link this tensor to its metadata file (e.g. labels).
    embedding.metadata_path = metadata
    # Saves a config file that TensorBoard will read during startup.
    projector.visualize_embeddings(tf.summary.FileWriter(LOG_DIR), config)